In [ ]:
import torch


In [ ]:
# kaggle autho key
from google.colab import files
files.upload()   # upload kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"raspingharecodes","key":"8adafb8a86743689a528dc5fc5ce79c2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#set the key

In [ ]:
!kaggle datasets download -d uraninjo/augmented-alzheimer-mri-dataset
#this downloads kaggle dataset

Dataset URL: https://www.kaggle.com/datasets/uraninjo/augmented-alzheimer-mri-dataset
License(s): GNU Lesser General Public License 3.0
 85% 322M/380M [00:00<00:00, 1.69GB/s]
100% 380M/380M [00:00<00:00, 1.69GB/s]


In [ ]:
!unzip augmented-alzheimer-mri-dataset.zip -d data/


Streaming output truncated to the last 5000 lines.
  inflating: data/OriginalDataset/NonDemented/31 (10).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (100).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (11).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (12).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (13).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (14).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (15).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (16).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (17).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (18).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (19).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (2).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (20).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (21).jpg  
  inflating: data/OriginalDataset/NonDemented/31 (22).jpg  
  inflating: data/OriginalDataset/NonDemented/31 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# set the  device for pytorch to run, should just return cuda

cuda


In [ ]:
from pathlib import Path

root = Path("data")

import os
def walk_through_dir(dir_path):
  '''Walks through dir_path returning its contents'''
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

#view the files
#can see directory through this

In [ ]:
walk_through_dir(root)
#visualiez the files
#use this to switch between datatsets

There are 2 directories and 0 images in 'data'
There are 4 directories and 0 images in 'data/AugmentedAlzheimerDataset'
There are 0 directories and 8960 images in 'data/AugmentedAlzheimerDataset/MildDemented'
There are 0 directories and 8960 images in 'data/AugmentedAlzheimerDataset/VeryMildDemented'
There are 0 directories and 6464 images in 'data/AugmentedAlzheimerDataset/ModerateDemented'
There are 0 directories and 9600 images in 'data/AugmentedAlzheimerDataset/NonDemented'
There are 4 directories and 0 images in 'data/OriginalDataset'
There are 0 directories and 896 images in 'data/OriginalDataset/MildDemented'
There are 0 directories and 2240 images in 'data/OriginalDataset/VeryMildDemented'
There are 0 directories and 64 images in 'data/OriginalDataset/ModerateDemented'
There are 0 directories and 3200 images in 'data/OriginalDataset/NonDemented'


In [ ]:
from torch.utils.data import DataLoader

# use this line to toggle set
image_path = root / "OriginalDataset"

#all the class directories
mildDemented = image_path / "MildDemented"
moderateDemented = image_path / "ModerateDemented"
nonDemented = image_path / "NonDemented"
veryMildDemented = image_path / "VeryMildDemented"



In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

# define a composition of transformations - only resize here
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128 pixels
])

#load the dataset using ImageFolder and apply data_transforms
full_dataset = ImageFolder(root=image_path, transform=data_transforms)

#80 20 test split
total_size = len(full_dataset)
test_size = int(0.2 * total_size)
dev_size = total_size - test_size

#get the datatsets from random splitting
dev_dataset, test_dataset = random_split(full_dataset,
                                  [dev_size, test_size],
                                  generator=torch.Generator().manual_seed(42))

print(f"Number of images in the dataset: {len(full_dataset)}")
print(f"Dev set: {len(dev_dataset)} Test set: {len(test_dataset)}")
print(f"Classes: {full_dataset.classes}")

Number of images in the dataset: 6400
Dev set: 5120 Test set: 1280
Classes: ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']


In [ ]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold

# K fold CV
num_splits = 5  #number of folds for cross-validation
kfold = KFold(n_splits=num_splits, shuffle=True, random_state=42)

BATCH_SIZE = 32 #32 size most optimal
# stores training and validation DataLoaders for each fold
train_loaders = []
val_loaders = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(dev_dataset)):
    print(f"Fold {fold+1}/{num_splits}")

    # subset for training testing validation
    train_subset = Subset(dev_dataset, train_idx)
    val_subset = Subset(dev_dataset, val_idx)

    #DataLoaders for current fold
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    #store DataLoaders
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

    print(f"Fold {fold+1}: Training has {len(train_loader)} batches (total {len(train_subset)} samples).")
    print(f"Fold {fold+1}: Validation has {len(val_loader)} batches (total {len(val_subset)} samples).")



Fold 1/5
Fold 1: Training has 128 batches (total 4096 samples).
Fold 1: Validation has 32 batches (total 1024 samples).
Fold 2/5
Fold 2: Training has 128 batches (total 4096 samples).
Fold 2: Validation has 32 batches (total 1024 samples).
Fold 3/5
Fold 3: Training has 128 batches (total 4096 samples).
Fold 3: Validation has 32 batches (total 1024 samples).
Fold 4/5
Fold 4: Training has 128 batches (total 4096 samples).
Fold 4: Validation has 32 batches (total 1024 samples).
Fold 5/5
Fold 5: Training has 128 batches (total 4096 samples).
Fold 5: Validation has 32 batches (total 1024 samples).


In [ ]:
import torch
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
from torch.utils.data import DataLoader
import numpy as np


#load pretrained model/processor
model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

num_labels = len(full_dataset.classes)


#custom dataset that works with JPG/PIL Images
#prepare images for the current model
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        #get PIL image (converted from JPG) and label
        image, label = self.dataset[idx]

        #handles PIL Images perfectly
        inputs = self.processor(images=image, return_tensors="pt")

        #rmove batch dimension, add label
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = label
        return inputs




In [ ]:
import evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#keep track of metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #sets accuracy accordingly
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

#stores all folds
fold_results = []
for i in range(len(train_loaders)):
  print(f"Fold {i+1}")

  train_dataset = ImageDataset(train_loaders[i].dataset, processor)
  val_dataset = ImageDataset(val_loaders[i].dataset, processor)

  model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # For changing the classification head
  )

  #trainer loop
  training_args = TrainingArguments(
    output_dir="./vit-finetuned", # output model runs
    eval_strategy="epoch", # every epoch to evaluate and save
    save_strategy="epoch",
    learning_rate=0.00002, # we used a low learning rate for convergence works better for ViT
    per_device_train_batch_size=16, # 16 batch size for optimal processing times
    per_device_eval_batch_size=16,
    num_train_epochs=5, # 5 epochs training
    weight_decay=0.01, #use regularization to prevent overfitting
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True, # or cuda
    remove_unused_columns=False,
  )

  accuracy_metric = evaluate.load("accuracy")


  # Initialize trainer
  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
  )

  trainer.train()

  results = trainer.evaluate()

  fold_results.append(results)
  print(f"Validation accuracy: {results['eval_accuracy']:.4f}")
  print(f"Validation loss: {results['eval_loss']:.4f}")
  print(f"Validation f1: {results['eval_f1']:.4f}")
  print(f"Validation precision: {results['eval_precision']:.4f}")
  print(f"Validation recall: {results['eval_recall']:.4f}")


Fold 1


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.683210,0.705078,0.729409,0.705078,0.706596
2,0.658000,0.367272,0.854492,0.845552,0.854492,0.848699
3,0.658000,0.250456,0.916016,0.916232,0.916016,0.915070
4,0.089700,0.212487,0.941406,0.942513,0.941406,0.941261
5,0.089700,0.172045,0.949219,0.950204,0.949219,0.949371


Validation accuracy: 0.9492
Validation loss: 0.1720
Validation f1: 0.9494
Validation precision: 0.9502
Validation recall: 0.9492
Fold 2


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.655921,0.746094,0.743081,0.746094,0.728608
2,0.616800,0.377268,0.849609,0.867186,0.849609,0.849940
3,0.616800,0.348240,0.889648,0.906042,0.889648,0.887308
4,0.073100,0.202973,0.940430,0.941080,0.940430,0.940282
5,0.073100,0.202777,0.943359,0.943879,0.943359,0.943184


Validation accuracy: 0.9434
Validation loss: 0.2028
Validation f1: 0.9432
Validation precision: 0.9439
Validation recall: 0.9434
Fold 5


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.664668,0.698242,0.714006,0.698242,0.676517
2,0.619100,0.404152,0.840820,0.862328,0.840820,0.836363
3,0.619100,0.286555,0.896484,0.898481,0.896484,0.895107
4,0.081700,0.260696,0.923828,0.926112,0.923828,0.923797
5,0.081700,0.206542,0.937500,0.937593,0.937500,0.937250


Validation accuracy: 0.9375
Validation loss: 0.2065
Validation f1: 0.9373
Validation precision: 0.9376
Validation recall: 0.9375


In [ ]:
#======== FINAL TRAINING ON ALL DEV DATA ================
final_train_dataset = ImageDataset(dev_dataset, processor)

#reinitialize model
final_model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

final_training_args = TrainingArguments(
    output_dir="./vit-final-model",
    eval_strategy="no",  #no validation during final training
    save_strategy="epoch",
    learning_rate=0.00002,
    per_device_train_batch_size=16,
    num_train_epochs=5,  #use same hyperparameters from best fold
    weight_decay=0.01,
    fp16=True,
    remove_unused_columns=False,
    report_to="none",
)

# Initialize Trainer
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=final_train_dataset,
    compute_metrics=compute_metrics,
)

final_trainer.train()

#======== TEST ON HELD-OUT TEST SET ================
print(f"\n{'='*50}")
print("FINAL EVALUATION ON TEST SET")
print(f"{'='*50}")

test_dataset_wrapped = ImageDataset(test_dataset, processor)

test_results = final_trainer.evaluate(test_dataset_wrapped)

print(f"\n🎯 FINAL TEST RESULTS:")
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test Loss: {test_results['eval_loss']:.4f}")
print(f"Test F1: {test_results['eval_f1']:.4f}")
print(f"Test Precision: {test_results['eval_precision']:.4f}")
print(f"Test Recall: {test_results['eval_recall']:.4f}")

# Save the final model
final_trainer.save_model("./vit-final-model-best")
processor.save_pretrained("./vit-final-model-best")
print("\nFinal model saved to './vit-final-model-best'")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.591900
1000,0.091800
1500,0.004300



FINAL EVALUATION ON TEST SET



🎯 FINAL TEST RESULTS:
Test Accuracy: 0.9734
Test Loss: 0.0836
Test F1: 0.9734
Test Precision: 0.9734
Test Recall: 0.9734

Final model saved to './vit-final-model-best'


In [ ]:
test_results

{'eval_loss': 0.08362022787332535,
 'eval_accuracy': 0.9734375,
 'eval_precision': 0.9734495236293105,
 'eval_recall': 0.9734375,
 'eval_f1': 0.9733616191444817,
 'eval_runtime': 6.0227,
 'eval_samples_per_second': 212.529,
 'eval_steps_per_second': 26.566,
 'epoch': 5.0}

For Augmented Dataset Transformer:

- Test Accuracy: 0.9848

- Test Loss: 0.0692

- Test F1: 0.9848

- Test Precision: 0.9848

- Test Recall: 0.9848


For Original Dataset Transformer
- Test Accuracy: 0.9789

- Test Loss: 0.0811

- Test F1: 0.9788

- Test Precision: 0.9789

- Test Recall: 0.9789

In [ ]:
# efficient net
import torch
from transformers import EfficientNetForImageClassification, EfficientNetImageProcessor, TrainingArguments, Trainer

# 1. EfficientNet Setup
efficientnet_model_name = "google/efficientnet-b0"
efficientnet_processor = EfficientNetImageProcessor.from_pretrained(efficientnet_model_name)
efficientnet_fold_results = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')





In [ ]:
# EFFICIENTNET
import torch
import numpy as np
from transformers import EfficientNetForImageClassification, EfficientNetImageProcessor, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#define compute metrics lcally ---
#function is now defined within this block, replacing the need for an external call.
def efficientnet_compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

#K-Fold CV
efficientnet_fold_results = []
#uses the train_loaders and val_loaders generated in Cell 22
for i in range(len(train_loaders)):
    print(f"\n--- EfficientNet Fold {i+1} ---")

    #use efficientNet-specific processor with the reusable ImageDataset class
    train_dataset = ImageDataset(train_loaders[i].dataset, efficientnet_processor)
    val_dataset = ImageDataset(val_loaders[i].dataset, efficientnet_processor)

    #load efficientNet model
    efficientnet_model = EfficientNetForImageClassification.from_pretrained(
        efficientnet_model_name,
        num_labels=num_labels, # Uses previous num_labels defined earlier
        ignore_mismatched_sizes=True
    ).to(device)

    # hyperparamters consistent
    efficientnet_training_args = TrainingArguments(
        output_dir=f"./efficientnet-finetuned/fold-{i+1}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=0.001, #vary this
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        fp16=True,
        remove_unused_columns=False,
        report_to="tensorboard",
        run_name=f"efficientnet-fold-{i+1}",
    )

    efficientnet_trainer = Trainer(
        model=efficientnet_model,
        args=efficientnet_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=efficientnet_compute_metrics, # USES LOCAL FUNCTION
    )

    efficientnet_trainer.train()
    results = efficientnet_trainer.evaluate()
    efficientnet_fold_results.append(results)
    print(f"EfficientNet Fold {i+1} Validation Accuracy: {results['eval_accuracy']:.4f}")

# final training on dev set
print(f"\n{'='*50}")
print("EfficientNet FINAL TRAINING ON DEV SET")
print(f"{'='*50}")



# Save the final model
efficientnet_trainer.save_model("./efficientnet-final-model-best")
efficientnet_processor.save_pretrained("./efficientnet-final-model-best")
print("EfficientNet Final model saved to './efficientnet-final-model-best'")


--- EfficientNet Fold 1 ---


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([4, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.578642,0.760742,0.769623,0.760742,0.742216
2,0.615300,0.606150,0.781250,0.841523,0.781250,0.782147
3,0.615300,0.229812,0.916016,0.918294,0.916016,0.916537
4,0.162600,0.231683,0.936523,0.940272,0.936523,0.937344
5,0.162600,0.177916,0.953125,0.953824,0.953125,0.952896
6,0.052400,0.105049,0.971680,0.971914,0.971680,0.971737
7,0.052400,0.124113,0.969727,0.971354,0.969727,0.969811
8,0.013800,0.111525,0.964844,0.964990,0.964844,0.964625
9,0.013800,0.100974,0.978516,0.978524,0.978516,0.978459
10,0.002800,0.108676,0.975586,0.976123,0.975586,0.975654


EfficientNet Fold 1 Validation Accuracy: 0.9785

--- EfficientNet Fold 2 ---


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([4, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.472689,0.807617,0.807317,0.807617,0.801848
2,0.574000,0.362965,0.878906,0.881265,0.878906,0.874973
3,0.574000,0.371070,0.871094,0.888847,0.871094,0.872307
4,0.163500,0.202134,0.940430,0.942676,0.940430,0.939984
5,0.163500,0.130636,0.960938,0.961464,0.960938,0.961008
6,0.055400,0.201749,0.950195,0.954877,0.950195,0.950625
7,0.055400,0.071669,0.980469,0.980664,0.980469,0.980446
8,0.007200,0.068551,0.983398,0.983547,0.983398,0.983423
9,0.007200,0.058765,0.980469,0.980714,0.980469,0.980473
10,0.001700,0.069666,0.977539,0.977799,0.977539,0.977494


EfficientNet Fold 4 Validation Accuracy: 0.9834

--- EfficientNet Fold 5 ---


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([4, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.791577,0.652344,0.748018,0.652344,0.666963
2,0.620700,0.286160,0.898438,0.900378,0.898438,0.898634
3,0.620700,0.215487,0.918945,0.919338,0.918945,0.919026
4,0.150600,0.203309,0.935547,0.938612,0.935547,0.935825
5,0.150600,0.303749,0.923828,0.931014,0.923828,0.924363
6,0.056700,0.211373,0.950195,0.950844,0.950195,0.949863
7,0.056700,0.193526,0.955078,0.955893,0.955078,0.954827
8,0.012800,0.132676,0.963867,0.965072,0.963867,0.963859
9,0.012800,0.098794,0.976562,0.976858,0.976562,0.976610
10,0.004500,0.104618,0.974609,0.974968,0.974609,0.974549


EfficientNet Fold 5 Validation Accuracy: 0.9766

EfficientNet FINAL TRAINING ON DEV SET


In [ ]:

#creates final training dataset
final_train_dataset = ImageDataset(dev_dataset, efficientnet_processor)
#loads pre trained EfficientNEt model
final_efficientnet_model = EfficientNetForImageClassification.from_pretrained(
    efficientnet_model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
).to(device)

#creates an instance of TrainingArguments to define all hyperparameters
final_efficientnet_training_args = TrainingArguments(
    output_dir="./efficientnet-final-model",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=0.001,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    remove_unused_columns=False,
    report_to="none",
)


#handles main training and eval loop
final_efficientnet_trainer = Trainer(
    model=final_efficientnet_model,
    args=final_efficientnet_training_args,
    train_dataset=final_train_dataset,
    compute_metrics=efficientnet_compute_metrics, # NOW USES LOCAL FUNCTION
)
final_efficientnet_trainer.train()

#FINAL eVAL ON HELD-OUT TEST SET
print(f"\n{'='*50}")
print("EfficientNet FINAL EVALUATION ON TEST SET")
print(f"{'='*50}")
test_dataset_wrapped = ImageDataset(test_dataset, efficientnet_processor)
efficientnet_test_results = final_efficientnet_trainer.evaluate(test_dataset_wrapped)

print(f"\n🎯 EfficientNet FINAL TEST RESULTS:")
print(f"Test Accuracy: {efficientnet_test_results['eval_accuracy']:.4f}")
print(f"Test Loss: {efficientnet_test_results['eval_loss']:.4f}")
print(f"Test F1: {efficientnet_test_results['eval_f1']:.4f}")
print(f"Test Precision: {efficientnet_test_results['eval_precision']:.4f}")
print(f"Test Recall: {efficientnet_test_results['eval_recall']:.4f}")


#save the final model
final_trainer.save_model("./effNet-final-model-best")
processor.save_pretrained("./effNet-final-model-best")
print("\nFinal model saved to './effNet-final-model-best'")

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([4, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.599100
1000,0.138100
1500,0.023700



EfficientNet FINAL EVALUATION ON TEST SET



🎯 EfficientNet FINAL TEST RESULTS:
Test Accuracy: 0.9867
Test Loss: 0.0549
Test F1: 0.9867
Test Precision: 0.9868
Test Recall: 0.9867

Final model saved to './effNet-final-model-best'


For Augmented Dataset EfficientNet:

- Test Accuracy: 0.996

- Test Loss: 0.0189

- Test F1: 0.996

- Test Precision: 0.996

- Test Recall: 0.996


For Original Dataset EfficientNet:
- Test Accuracy: 0.9867

- Test Loss: 0.0549

- Test F1: 0.9867

- Test Precision: 0.9868

- Test Recall: 0.9867